## Cobot Projectile Obstacle Avoidance
Work Items:
1. Obstacle generator
2. YOLO image segmentation
3. Cobot control and DQN

In [5]:
import pybullet as p
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import cobot_ai4robotics

In [6]:
# Basic code to open the GUI environment window.
# https://www.gymlibrary.dev/content/environment_creation/ - make a custom gym environment.

env = gym.make("cobot_ai4robotics", apply_api_compatibility=True, renders=True, isDiscrete=False)
env.reset()
while True:
    ob, reward, done, _, info = env.step(env.action_space.sample()) # Things will not move unless you call env.step.
    if done:
        break
env.close()

Contact by banana no. 8 at point (0.09 0.09 0.69) on KUKA.
Contact by banana no. 8 at point (0.07 0.10 0.65) on KUKA.
Contact by banana no. 8 at point (0.05 0.09 0.65) on KUKA.
Contact by banana no. 8 at point (0.04 0.08 0.65) on KUKA.
Contact by banana no. 15 at point (0.15 0.09 0.66) on KUKA.
Contact by banana no. 15 at point (0.14 0.10 0.64) on KUKA.
Contact by banana no. 15 at point (0.14 0.10 0.64) on KUKA.
Contact by banana no. 15 at point (0.13 0.10 0.64) on KUKA.
Contact by banana no. 1 at point (0.15 -0.08 0.93) on KUKA.
Contact by banana no. 1 at point (0.10 -0.11 0.88) on KUKA.
Contact by banana no. 1 at point (0.18 -0.03 0.84) on KUKA.
Contact by banana no. 1 at point (0.17 -0.04 0.80) on KUKA.
Contact by banana no. 1 at point (0.15 -0.07 0.77) on KUKA.
Contact by banana no. 3 at point (0.27 -0.02 1.36) on KUKA.
Contact by banana no. 3 at point (0.69 -0.30 1.39) on KUKA.
Contact by banana no. 4 at point (0.39 -0.34 1.55) on KUKA.
Contact by banana no. 4 at point (0.39 -0.34

error: Not connected to physics server.